# Basic example
This is a basic notebook, used only to detect and track objects

In [ ]:
import cv2
import glob
import os
import sys
import numpy as np
import pickle 

os.environ["CUDA_VISIBLE_DEVICES"]="1"

from random import randint
import matplotlib.pyplot as plt

import socket
import json

from time import time
from cav.objects import Object, BoundingBox, ObjectType
from cav.parameters import Parameters

from cav.visualization import Map, plotBoxes, bsmImg

import tensorflow as tf

# Deep sort imports
from deep_sort import nn_matching
from deep_sort.tracker import Tracker
from deep_sort.detection import Detection
%matplotlib inline 


## Deep sort

In [ ]:
from helper import ImageEncoder, create_box_encoder, extract_image_patch

In [ ]:
ENCODER_PATH = "./deep_sort_network/mars-small128.pb"
ENCODER_BATCH_SIZE = 32
ENCODER_INPUT_NAME = "images"
ENCODER_OUTPUT_NAME = "features"

image_encoder = ImageEncoder(ENCODER_PATH, ENCODER_INPUT_NAME, ENCODER_OUTPUT_NAME)
encoder = create_box_encoder(ENCODER_PATH, batch_size=32)

## Create tracker

In [ ]:
max_cosine_distance = 0.2
nn_budget = 100

metric = nn_matching.NearestNeighborDistanceMetric(
    "cosine", max_cosine_distance, nn_budget)

### Create object detector

In [ ]:
from cav.detection import ObjectDetector
import cv2
#MODEL_PATH = '../../ObjectDetection/models/frcnn_old/inference/saved_model/'
MODEL_PATH = '../../ObjectDetection/models/frcnn/inference/saved_model/'

od = ObjectDetector(MODEL_PATH)

#### Parameters

In [ ]:
FRAME_FOLDER = './MoveOver/PAI81_66/frames/frames_raw/'
SAVE_LOG = None #### Saves logs with all detected objects (path to file or none)
#SAVE_LOG = 'test_log_20201028.csv'

CUT_UPPER = 0 # Delete first X rows
#SAVE_DETECTIONS = None #### Saves everything in a dictionary
SAVE_DETECTIONS = './MoveOver/PAI81_66/detections.p'

In [ ]:
cap = cv2.VideoCapture('/data/MoveOver/Videos/PA/D8_81-66-2021-02-23 FHWA.mp4')
#cap = cv2.VideoCapture('/data/MoveOver/Videos/PA/8 D8_30-PA 501-2021-03-02 FHWA.mp4')

tracker = Tracker(metric)

objects = []

results = []
colors = {}


save_detections = {}    

# Delete old files
filenames = glob.glob(os.path.join(FRAME_FOLDER, '*.jpg'))
for f in filenames:
    os.remove(f)




i = 0
t = time()

while cap.isOpened():
    video_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
    video_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) - CUT_UPPER     
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter('Test1_output.avi',cv2.VideoWriter_fourcc(*'DIVX'), fps, (video_x, video_y))

    frame_timeStamp = i/fps

    t2 = time() - t
    sys.stdout.write('{} frames done in {:.1f} seconds ({:.2f} frames/sec)    \r'.format(
        i, t2, i/t2))                   
    i += 1
    
    ret, image = cap.read()

    if CUT_UPPER > 0:
        image = image[CUT_UPPER:, :, :]
    
    boxes, scores, classes = od.detect(image, timestamp=frame_timeStamp)
    if SAVE_DETECTIONS is not None:
        save_detections[i] = (boxes, scores, classes)
        
        
    if len(boxes) >= 1:
        boxes_array = [[box.xLeft, box.yTop, box.xRight - box.xLeft, box.yBottom - box.yTop] for box in boxes]
        boxes_array = np.array(boxes_array)
        bgr_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        features = encoder(bgr_image, boxes_array)
        detections = []

        for box, score, objClass, f_vector in zip(boxes, scores, classes, features):
            detection = Detection(
                [box.xLeft, box.yTop, box.xRight - box.xLeft, box.yBottom - box.yTop], #BBox
                score, f_vector,
                objClass
            )
            detection.bbox = box
            detections.append(detection)

        tracker.predict()
        tracker.update(detections)   
    else:
        tracker.predict()

    plotboxes = []
    plotcolors = []
    objects = []

    if len(tracker.tracks) >= 1:
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue

            if False:
                bbox = track.to_tlwh()
                results.append([
                    i, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])


            obj = track.trackedObject

            if obj is not None:
                if obj.color is None:
                    obj.color = (randint(0, 255), randint(0, 255), randint(0, 255))       
                plotbox = obj.bboxes[-1]
                plotbox.trackId = track.track_id
                plotboxes.append(plotbox)
                plotcolors.append(obj.color)
                objects.append(obj)


    if len(plotboxes) >= 1:
        vid = plotBoxes(image, plotboxes, colors=plotcolors)
    else:
        vid = image.copy()
        
    cv2.imwrite(os.path.join(FRAME_FOLDER, 'im_{}.jpg'.format(str(i).zfill(5))), vid)
    #out.write(vid)            
         
    if i > fps * 60 * 45:
        break
                        
t = time() - t                             
print('\n\n{} frames done in {:.1f} seconds ({:.2f} frames/sec)'.format(
    i, t, i/t))                             
cap.release()

if SAVE_DETECTIONS is not None:
    pickle.dump(save_detections, open(SAVE_DETECTIONS,'wb'))



In [ ]:
print ('({}x{}), {:.1f} fps'.format(video_x, video_y, fps))

# Generate Video

In [ ]:
from helper import generate_video
generate_video(fps = fps, outputFile='output_video_test2.avi')

# Use saved detections - test

In [ ]:
save_detections = pickle.load(open(SAVE_DETECTIONS,'rb'))
save_detections[1]

In [ ]:
cap = cv2.VideoCapture('/data/MoveOver/Videos/PA/D8_81-66-2021-02-23 FHWA.mp4')

tracker = Tracker(metric)

objects = []

results = []
colors = {}



# Delete old files
filenames = glob.glob(os.path.join(FRAME_FOLDER, '*.jpg'))
for f in filenames:
    os.remove(f)




i = 0
t = time()

while cap.isOpened():
    video_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
    video_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) - CUT_UPPER     
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter('Test1_output.avi',cv2.VideoWriter_fourcc(*'DIVX'), fps, (video_x, video_y))


    t2 = time() - t
    sys.stdout.write('{} frames done in {:.1f} seconds ({:.2f} frames/sec)    \r'.format(
        i, t2, i/t2))                   
    i += 1
    
    ret, image = cap.read()

    if CUT_UPPER > 0:
        image = image[CUT_UPPER:, :, :]
    
    #boxes, scores, classes = od.detect(image)
    boxes, scores, classes = save_detections[i]    

    if len(boxes) >= 1:
        boxes_array = [[box.xLeft, box.yTop, box.xRight - box.xLeft, box.yBottom - box.yTop] for box in boxes]
        boxes_array = np.array(boxes_array)
        bgr_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        features = encoder(bgr_image, boxes_array)
        detections = []

        for box, score, objClass, f_vector in zip(boxes, scores, classes, features):
            detection = Detection(
                [box.xLeft, box.yTop, box.xRight - box.xLeft, box.yBottom - box.yTop], #BBox
                score, f_vector,
                objClass
            )
            detection.bbox = box
            detections.append(detection)

        tracker.predict()
        tracker.update(detections)   
    else:
        tracker.predict()

    plotboxes = []
    plotcolors = []
    objects = []

    if len(tracker.tracks) >= 1:
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue

            if False:
                bbox = track.to_tlwh()
                results.append([
                    i, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])


            obj = track.trackedObject

            if obj is not None:
                if obj.color is None:
                    obj.color = (randint(0, 255), randint(0, 255), randint(0, 255))       
                plotbox = obj.bboxes[-1]
                plotbox.trackId = track.track_id
                plotboxes.append(plotbox)
                plotcolors.append(obj.color)
                objects.append(obj)


    if len(plotboxes) >= 1:
        vid = plotBoxes(image, plotboxes, colors=plotcolors)
    else:
        vid = image.copy()
        
    cv2.imwrite(os.path.join(FRAME_FOLDER, 'im_{}.jpg'.format(str(i).zfill(5))), vid)
    #out.write(vid)            
         
    if i > fps * 60 * 30:
        break
                        
t = time() - t                             
print('\n\n{} frames done in {:.1f} seconds ({:.2f} frames/sec)'.format(
    i, t, i/t))                             
cap.release()
